In [ ]:
import igl # Library to load meshes and perform operations on them
import meshplot as mp # Library to visualize meshes and point clouds
import vedo as vd # Library to visualize meshes and point clouds
import polyscope as ps # Library to visualize meshes
import numpy as np # Library to perform operations on matrices
import os # Library to perform operations on files and directories
import matplotlib.pyplot as plt
import math
# Importing the classes and functions from the visualization folder
vd.settings.default_backend = 'k3d'

# Directory path
dir_path = os.getcwd()

In [ ]:
original_path = "C:/Users/aikyna/Desktop/Projects/Meshes_with_Spherical_faces/Darboux_Transformation_Data/Tri/Original_tri.obj"
dt_path = "C:/Users/aikyna/Desktop/Projects/Meshes_with_Spherical_faces/Darboux_Transformation_Data/Tri/DT1_tri.obj"
spheres_path = "C:/Users/aikyna/Desktop/Projects/Meshes_with_Spherical_faces/Darboux_Transformation_Data/Tri/sph_tri.dat"
abs_sph_path = "C:/Users/aikyna/Desktop/Projects/Meshes_with_Spherical_faces/Darboux_Transformation_Data/Tri/Omega_tri.dat"
my_sphere_path = "C:/Users/aikyna/Desktop/Projects/Meshes_with_Spherical_faces/Darboux_Transformation_Data/Tri/Alisher_Omega_tri.dat"
my_spheres_path = "C:/Users/aikyna/Desktop/Projects/Meshes_with_Spherical_faces/Darboux_Transformation_Data/Tri/Alisher_sph_tri.dat"

In [ ]:
v, f = igl.read_triangle_mesh(original_path)
p = mp.plot(v, f, return_plot=True)


In [ ]:
v, f = igl.read_triangle_mesh(dt_path)
V = len(v) # Number of vertices
F = len(f) # Number of faces

abs_sphere = np.loadtxt(my_sphere_path,  dtype=float)
center_abs, rad_abs = abs_sphere[0:3], abs_sphere[3]
print(center_abs, rad_abs)

p = mp.plot(v, f, return_plot=True)
p.add_points(np.array([center_abs]), shading={"point_color": "green", "point_size": 1.0})
p

In [ ]:
normalized = np.ones(3)
normalized /= np.linalg.norm(normalized)
normals_per_face = -igl.per_face_normals(v, f, normalized)


circ_centers = np.zeros((F, 3))
circ_rads = np.zeros(F)

sphere_centers = np.zeros((F, 3))
sphere_rads = np.zeros(F)

matrix_a = np.zeros((3,3))
vector_b = np.zeros(3)
matrix_a[0] = np.array([1, 1, 1])
vector_b[0] = 1

count = 0

t_signs = np.zeros(F)
for f_ind in range(F):
    v0, v1, v2 = v[f[f_ind][0]], v[f[f_ind][1]], v[f[f_ind][2]]
    matrix_a[1] = np.array([-2 * np.dot(v0, v0) + 2 * np.dot(v0, v1), -2 * np.dot(v1,v0) + 2 * np.dot(v1,v1), -2 * np.dot(v2,v0) + 2 * np.dot(v2,v1)])
    vector_b[1] = np.dot(v1, v1) - np.dot(v0, v0)
    matrix_a[2] = np.array([2 * np.dot(v0, v2) - 2 * np.dot(v0, v1), 2 * np.dot(v1,v2) - 2 * np.dot(v1,v1), 2 * np.dot(v2,v2) - 2 * np.dot(v2,v1)])
    vector_b[2] = np.dot(v2, v2) - np.dot(v1, v1)
    mu0, mu1, mu2 = np.linalg.solve(matrix_a, vector_b)
    circ_centers[f_ind] = mu0 * v0 + mu1 * v1 + mu2 * v2
    circ_rads[f_ind] = np.linalg.norm(v0 - circ_centers[f_ind])

    t = (circ_rads[f_ind] ** 2 + rad_abs ** 2 - np.dot(circ_centers[f_ind] - center_abs, circ_centers[f_ind] - center_abs)) / (-2 * np.dot(circ_centers[f_ind] - center_abs, normals_per_face[f_ind]))
    if (t < 0):
        count += 1
        t_signs[f_ind] = -1
    else:
        t_signs[f_ind] = 1
    sphere_centers[f_ind] = circ_centers[f_ind] - t * normals_per_face[f_ind]
    sphere_rads[f_ind] = np.sqrt(circ_rads[f_ind] ** 2 + t ** 2)

print(count)
p = mp.plot(v, f, return_plot=True, c = t_signs)

In [ ]:
 #print(np.dot(circ_centers[26] - t * normals_per_face[26] - center_abs, circ_centers[26] - t * normals_per_face[26] - center_abs) - rad_abs ** 2 - t ** 2 - circ_rads[26] ** 2)

In [ ]:
ps.init()
ps.remove_all_structures()
 
mesh = ps.register_surface_mesh("Mesh", v, f, smooth_shade=True)
mesh.add_scalar_quantity("Mean curvature", t_signs, defined_on='faces')

for _ in range(10):
    i = np.random.randint(0, F-1)
    c = sphere_centers[i] 
    sphere = ps.register_point_cloud(f"sphere_c{i}", np.array([c]), enabled=True, color=(0,0,0), transparency=0.3)
    sphere.set_radius(sphere_rads[i], relative=False)

abs_sphere = ps.register_point_cloud(f"abs_sphere", np.array([center_abs]), enabled=True, color=(0,0,256), transparency=0.9)
abs_sphere.set_radius(rad_abs, relative=False)

ps.show()

In [ ]:
orth = np.zeros(F)
for f_ind in range(F):
    v0, v1, v2 = v[f[f_ind][0]], v[f[f_ind][1]], v[f[f_ind][2]]
    err1, err2, err3 = abs(np.linalg.norm(v0 - sphere_centers[f_ind]) - sphere_rads[f_ind]), abs(np.linalg.norm(v1 - sphere_centers[f_ind]) - sphere_rads[f_ind]), abs(np.linalg.norm(v2 - sphere_centers[f_ind]) - sphere_rads[f_ind])
    if (err1 > 10e-10 or err2 > 10e-10 or err3 > 10e-10):
        print("error #", f_ind)
    orth[f_ind] = abs(np.dot(sphere_centers[f_ind] - center_abs, sphere_centers[f_ind] - center_abs) - sphere_rads[f_ind] ** 2 - rad_abs ** 2)
print(max(orth))


In [ ]:
file_export = open(my_spheres_path, "w")

# Export face neighbors to .dat file
for f_ind in range(F):
    for j in range(3):
        file_export.write(f"{sphere_centers[f_ind][j]}\t")
    file_export.write(f"{sphere_rads[f_ind]}\n")

file_export.close()